In [1]:
@time using Plots
@time using Images
@time using ColorSchemes

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


132.619609 seconds (26.48 M allocations: 1.391 GiB, 0.72% gc time)


┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1260


149.196535 seconds (8.95 M allocations: 474.319 MiB, 0.20% gc time)
  0.000701 seconds (581 allocations: 28.328 KiB)


## Auxilliary subroutines for SIR simulation

In [2]:
function neighbourIndices(nType::Int64, rad::Int64)
    indices = []
    if(nType == 0)
        indices = vcat([[i, 0] for i in -rad:rad], [[0, j] for j in -rad:rad])
    elseif(nType == 1)
        indices = [[i, j] for i in -rad:rad for j in -rad:rad]
    elseif(nType == 2)
        indices = [[i, j] for j in -rad:rad for i in (abs(j) - rad):(rad - abs(j))]
    end
    
    indices = [CartesianIndex(ind...) for ind in setdiff(indices, [[0,0]])]
end


neighbourIndices (generic function with 1 method)

## Initial Conditions

In [3]:
function setInitConditions(grid, init, tauI, tauR, indices)
    if(init[1] == "random")
        for index in indices
            if(rand() < init[2])
                grid[index] = tauI+1
            end
        end
        grid[init[3], init[4]] = 1
    
    elseif(init[1] == "assorted")
        for index in indices
            if(rand() < init[2])
                grid[index] = 1
            elseif(rand() < init[3])
                grid[index] = tauI+1
            end
        end
        
    elseif(init[1] == "single")
        grid[init[2], init[3]] = 1
        
    elseif(init[1] == "corners")
        grid[rad + 2, rad + 2] = 1
        grid[rad + 2, n - rad - 2] = 1
        grid[n - rad - 2, rad + 2] = 1
        grid[n - rad - 2, n - rad - 2]  = 1
         
    elseif(init[1] == "custom")
        grid[init[2], init[3]] = 1
        grid[init[4], init[5]] = tauI + 1
    end
    
    return
end

setInitConditions (generic function with 1 method)

## Simulation

In [65]:
function SIRSmodel(n::Int64, tauI::Int64, tauR::Int64, nType::Int64, rad::Int64, nsteps::Int64, anim::Bool, colbar::Bool, fname, init)
    grid = zeros(Int64, (n, n)) #2D grid
    nData = zeros(Int64, (nsteps, 3)) #S,I,R count
    frames = [] #to store grid for display
    
    infp = 0 #infection probability
    tau0 = tauR + tauI #complete disease cycle 
    relNeighbour = neighbourIndices(nType, rad) #relative index of neighbours
    isInfected(x) = 1 <= x <= tauI #infected check
    indices = [CartesianIndex(i,j) for i in (rad+1):(n-rad) for j in (rad+1):(n-rad)]

    #initial conditions
    setInitConditions(grid, init, tauI, tauR, indices)
    
    #boundary conditions; fixed
    grid[1:rad, :] .= tauI + 1
    grid[(n-rad+1):n, :] .= tauI + 1
    grid[:, 1:rad] .= tauI + 1
    grid[:, (n-rad+1):n] .= tauI + 1   
    
    prev = copy(grid) #copy of the grid
    push!(frames, copy(grid))
    
    #initial count
    nData[1, 1] = count(x -> x==0, grid)
    nData[1, 2] = count(x -> 1 <= x <= tauI, grid)
    nData[1, 3] = count(x -> tauI < x <= tau0, grid)
    
    for k in 2:nsteps
        nData[k, :] = nData[k-1, :]
        
        for index in indices
            if(prev[index] == 0) #susceptible
                infp = count(isInfected.(prev[relNeighbour .|> x-> index+x]))/length(relNeighbour)
                
                if(0<infp) #infection
                    grid[index] += 1
                    nData[k, 1] -= 1
                    nData[k, 2] += 1
                end
                
            elseif(prev[index] < tau0) #infected/refractory
                grid[index] += 1
                
                if(grid[index] == tauI + 1) #transition to R
                    nData[k, 2] -= 1
                    nData[k, 3] += 1
                end
            
            elseif(prev[index] == tau0) #transition to S
                grid[index] = 0
                nData[k, 3] -= 1
                nData[k, 1] += 1
            end
        end
        
        prev = copy(grid)
        push!(frames, copy(grid))
    end
    
    #color gradient
    colors = vcat([RGB{N0f8}(0, 0, 0)], range(RGB{N0f8}(0.5,0.0,0.0), stop=RGB{N0f8}(0.862,0,0), length= tauI),range(RGB{N0f8}(1,0.458,0.0), stop=RGB{N0f8}(1, 1, 0.196), length=tauR))
    
    if(colbar) #with colorbar
        animGIF = @animate for i = 1:nsteps 
            heatmap(frames[i], c = palette(colors, tau0+1), clim = (0, tau0))
        end
        
    else #without colorbar
        cmap = Dict(zip(0:tau0+1, colors))
        animGIF = @animate for i = 1:nsteps 
            plot([cmap[p] for p in frames[i]])
        end
    end
        
    gif(animGIF, fname, fps = 15)
    
    return nData
end     
    

SIRSmodel (generic function with 1 method)

In [64]:
#function SIRSmodel(n, tauI, tauR, nType, rad, nsteps, anim, colorbar, init)
nsteps = 200
s = 150
num = 1
@time data = SIRSmodel(s, 3, 5, 2, num, nsteps, true, false, "outputs2/2SIRS"*string(num)*".gif", ["single", s÷2, s÷2])
x = 1:nsteps
plot(x, data, label = ["Susceptible" "Infected" "Refractory"], lw = 2, grid = true)
savefig("outputs2/plot2"*string(num)*".png")

io = open("outputs2/data2"*string(num)*".txt", "w")
println(io, data[:, 1])
close(io)

 18.774421 seconds (59.97 M allocations: 2.619 GiB, 2.09% gc time)


┌ Info: Saved animation to 
│   fn = /home/akshay/code-repo/ComplexSystems/outputs2/2SIRS1.gif
└ @ Plots /home/akshay/.julia/packages/Plots/uCh2y/src/animation.jl:104


## Collect data version

In [147]:
function SIRSmodelCollect(n::Int64, tauI::Int64, tauR::Int64, nType::Int64, rad::Int64, nsteps::Int64, anim::Bool, colbar::Bool, fname, init)
    grid = zeros(Int64, (n + 2*rad, n + 2*rad)) #2D grid
    nData = zeros(Int64, (nsteps, 3)) #S,I,R count
    frames = [] #to store grid for display
    
    infp = 0 #infection probability
    tau0 = tauR + tauI #complete disease cycle 
    isInfected(x) = 1 <= x <= tauI #infected check
    
    relNeighbour = neighbourIndices(nType, rad) #relative index of neighbours
    indices = [CartesianIndex(i,j) for i in (rad+1):(n+rad) for j in (rad+1):(n+rad)]
    
    #initial conditions
    setInitConditions(grid, init, tauI, tauR, indices)
    
    #boundary conditions; fixed
    grid[1:rad, :] .= tauI + 1
    grid[(n+rad+1):(n+2*rad), :] .= tauI + 1
    grid[:, 1:rad] .= tauI + 1
    grid[:, (n+rad+1):(n+2*rad)] .= tauI + 1   
    
    prev = copy(grid) #copy of the grid
    timesteps = 1 #timesteps passed
    push!(frames, copy(grid))
    
    #initial count
    nData[1, 1] = count(x -> x==0, grid)
    nData[1, 2] = count(x -> 1 <= x <= tauI, grid)
    nData[1, 3] = count(x -> tauI < x <= tau0, grid)
    
    for k in 2:nsteps
        nData[k, :] = nData[k-1, :]
        
        for index in indices
            if(prev[index] == 0) #susceptible
                infp = count(isInfected.(prev[relNeighbour .|> x-> index+x]))/length(relNeighbour)
                
                if(0<infp) #infection
                    grid[index] += 1
                    nData[k, 1] -= 1
                    nData[k, 2] += 1
                end
                
            elseif(prev[index] < tau0) #infected/refractory
                grid[index] += 1
                
                if(grid[index] == tauI + 1) #transition to R
                    nData[k, 2] -= 1
                    nData[k, 3] += 1
                end
            
            elseif(prev[index] == tau0) #transition to S
                grid[index] = 0
                nData[k, 3] -= 1
                nData[k, 1] += 1
            end
        end
        
        prev = copy(grid)
        push!(frames, copy(grid))
        timesteps += 1
        
        if(nData[k, 1] == n^2 && k>2)  
            break
        end
        
    end
    
    #color gradient
    colors = vcat([RGB{N0f8}(0, 0, 0)], range(RGB{N0f8}(0.5,0.0,0.0), stop=RGB{N0f8}(0.862,0,0), length= tauI),range(RGB{N0f8}(1,0.458,0.0), stop=RGB{N0f8}(1, 1, 0.196), length=tauR))
    
    if(colbar) #with colorbar
        animGIF = @animate for i = 1:timesteps 
            heatmap(frames[i], c = palette(colors, tau0+1), clim = (0, tau0))
        end
        
    else #without colorbar
        cmap = Dict(zip(0:tau0+1, colors))
        animGIF = @animate for i = 1:timesteps 
            plot([cmap[p] for p in frames[i]])
        end
    end
        
    gif(animGIF, fname, fps = 15)
    
    nData[timesteps:end, 1] .= nData[timesteps-1, 1]
    nData[timesteps:end, 2] .= nData[timesteps-1, 2]
    nData[timesteps:end, 3] .= nData[timesteps-1, 3]    
    return [nData, timesteps]
end     
    

SIRSmodelCollect (generic function with 2 methods)

In [152]:
S2 = []
I2 = []
R2 = []
numtime2 = []

0-element Array{Any,1}

In [167]:
#function SIRSmodel(n, tauI, tauR, nType, rad, nsteps, anim, colorbar, init)
nsteps = 300
s = 200
num = 15
@time data = SIRSmodelCollect(s, 3, 5, 2, num, nsteps, true, false, "outputs2/2SIRS"*string(num)*".gif", ["single", s÷2, s÷2])
push!(S2, minimum(data[1][:, 1]))
push!(I2, maximum(data[1][:, 2]))
push!(R2, maximum(data[1][:, 3]))
push!(numtime2, data[2])

x = 1:nsteps
plot(x, data[1][:,:], label = ["Susceptible" "Infected" "Refractory"], lw = 2, grid = true)
savefig("outputs2/plot2"*string(num)*".png")

io = open("outputs2/data2"*string(num)*".txt", "w")
println(io, data[1][:, 1])
close(io)

println(numtime2)

  7.666011 seconds (9.59 M allocations: 9.860 GiB, 9.05% gc time)
Any[211, 111, 78, 61, 51, 45, 40, 36, 34, 31, 30, 28, 27, 26, 25]


┌ Info: Saved animation to 
│   fn = /home/akshay/code-repo/ComplexSystems/outputs2/2SIRS15.gif
└ @ Plots /home/akshay/.julia/packages/Plots/uCh2y/src/animation.jl:104


In [172]:
print(S2,"\n\n" ,I2, "\n\n", R2, "\n\n",numtime2)

Any[36874, 33882, 31026, 28306, 25722, 23298, 20970, 18834, 16746, 14842, 13074, 11490, 10026, 8594, 7482]

Any[1182, 2338, 3466, 4566, 5638, 6706, 7718, 8734, 9646, 10578, 11482, 12454, 13334, 14074, 14998]

Any[2770, 5490, 8158, 10774, 13338, 15874, 18330, 20766, 23074, 25378, 27630, 29926, 32106, 34122, 36298]

Any[211, 111, 78, 61, 51, 45, 40, 36, 34, 31, 30, 28, 27, 26, 25]